# Importing Libraries

First we import the libraries necessary to analyze the data


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import utils as m

In [2]:
#load the cleaned data

incidents = pd.read_csv("../data/incidents_cleaned.csv", low_memory=False)
poverty = pd.read_csv("../data/poverty_cleaned.csv", low_memory=False)
districts = pd.read_csv("../data/districts_cleaned.csv", low_memory=False)

# and load our external data
population = pd.read_csv("../data/Population by Age and Sex - US, States, Counties.csv", low_memory=False)


In [3]:
incidents.info()
incidents = incidents.drop(
    columns=['participant_age1', 'participant_age_group1', 'participant_gender1', 'state_house_district',
             'state_senate_district', 'address'])
incidents.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216389 entries, 0 to 216388
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   date                       216389 non-null  object 
 1   state                      216389 non-null  object 
 2   city_or_county             216389 non-null  object 
 3   address                    201681 non-null  object 
 4   latitude                   208488 non-null  float64
 5   longitude                  208488 non-null  float64
 6   congressional_district     204475 non-null  float64
 7   state_house_district       180606 non-null  float64
 8   state_senate_district      186659 non-null  float64
 9   participant_age1           147016 non-null  float64
 10  participant_age_group1     174409 non-null  object 
 11  participant_gender1        180045 non-null  object 
 12  min_age_participants       147020 non-null  float64
 13  avg_age_participants       14

,date,state,city_or_county,latitude,longitude,congressional_district,min_age_participants,avg_age_participants,max_age_participants,n_participants_child,...,n_injured,n_arrested,n_unharmed,n_participants,notes,incident_characteristics1,incident_characteristics2,year,month,day
0,2015-05-02,Indiana,Indianapolis,39.8322,-86.2492,7.0,19.0,19.0,19.0,0.0,...,1,0,0,1,teen wounded while walking - security guard at...,shot - wounded/injured,NaN,2015,5,2
1,2017-04-03,Pennsylvania,Kane,41.6645,-78.7856,5.0,62.0,62.0,62.0,0.0,...,0,0,0,1,shot self after accident,"shot - dead (murder, accidental, suicide)",suicide^,2017,4,3
2,2016-11-05,Michigan,Detroit,42.4190,-83.0393,14.0,NaN,NaN,NaN,NaN,...,1,0,1,2,1 inj.,shot - wounded/injured,NaN,2016,11,5
3,2016-10-15,District of Columbia,Washington,38.9030,-76.9820,1.0,NaN,NaN,NaN,0.0,...,1,0,0,2,NaN,shot - wounded/injured,NaN,2016,10,15
4,2014-01-18,North Carolina,Wayne County,35.1847,-77.9527,13.0,65.0,65.0,65.0,0.0,...,1,0,0,1,NaN,shot - wounded/injured,home invasion,2014,1,18


In [4]:
incidents['incident_characteristics1'].unique()

value_counts = incidents['incident_characteristics1'].value_counts(dropna=False)
print(value_counts)

incident_characteristics1
shot - wounded/injured                                                                                  83570
shot - dead (murder, accidental, suicide)                                                               42821
non-shooting incident                                                                                   39696
shots fired - no injuries                                                                               31576
institution/group/business                                                                               4029
armed robbery with injury/death and/or evidence of dgu found                                             3444
tsa action                                                                                               2639
possession (gun(s) found during commission of other crimes)                                              1106
brandishing/flourishing/open carry/lost/found                                                 

In [5]:
# Define the categories
category_mapping = {
    'Shooting Casualty Incidents': [
        'shot - wounded/injured', 'shot - dead (murder, accidental, suicide)',
        # Add other characteristics from this category
    ],
    'Non-Fatal Shooting Incidents': [
        'shots fired - no injuries', 'shots fired, no action (reported, no evidence found)',
        # Add other characteristics from this category
    ],
    'Non-Shooting Gun-Related Incidents': [
        'institution/group/business', 'possession (gun(s) found during commission of other crimes)',
        # Add other characteristics from this category
    ],
    # 'Unspecified' category includes NaN values
    'Unspecified': [np.nan]
}

# Function to map incident characteristics to category indices
def map_to_category(characteristic):
    for category, characteristics in category_mapping.items():
        if pd.isna(characteristic) and pd.isna(characteristics[0]):
            return list(category_mapping.keys()).index(category) + 1  # Adding 1 to start indexing from 1
        elif characteristic in characteristics:
            return list(category_mapping.keys()).index(category) + 1  # Adding 1 to start indexing from 1
    return 0  # 0 represents 'Unspecified or Miscellaneous Incidents'

# Apply the mapping function to create a new column 'category_index'
incidents['category_index'] = incidents['incident_characteristics1'].apply(map_to_category)
incidents.head()

,date,state,city_or_county,latitude,longitude,congressional_district,min_age_participants,avg_age_participants,max_age_participants,n_participants_child,...,n_arrested,n_unharmed,n_participants,notes,incident_characteristics1,incident_characteristics2,year,month,day,category_index
0,2015-05-02,Indiana,Indianapolis,39.8322,-86.2492,7.0,19.0,19.0,19.0,0.0,...,0,0,1,teen wounded while walking - security guard at...,shot - wounded/injured,NaN,2015,5,2,1
1,2017-04-03,Pennsylvania,Kane,41.6645,-78.7856,5.0,62.0,62.0,62.0,0.0,...,0,0,1,shot self after accident,"shot - dead (murder, accidental, suicide)",suicide^,2017,4,3,1
2,2016-11-05,Michigan,Detroit,42.4190,-83.0393,14.0,NaN,NaN,NaN,NaN,...,0,1,2,1 inj.,shot - wounded/injured,NaN,2016,11,5,1
3,2016-10-15,District of Columbia,Washington,38.9030,-76.9820,1.0,NaN,NaN,NaN,0.0,...,0,0,2,NaN,shot - wounded/injured,NaN,2016,10,15,1
4,2014-01-18,North Carolina,Wayne County,35.1847,-77.9527,13.0,65.0,65.0,65.0,0.0,...,0,0,1,NaN,shot - wounded/injured,home invasion,2014,1,18,1


In [6]:
# we can now drop the incident_characteristics1, incident_characteristics2 and notes columns

incidents = incidents.drop(columns=['incident_characteristics1', 'incident_characteristics2', 'notes'])

In [7]:
incidents_and_poverty = pd.merge(incidents, poverty, on=['state', 'year'], how='inner')
incidents_and_poverty = incidents_and_poverty[['state', 'year', 'date', 'povertyPercentage']]


In [8]:
population = population[(population['Countyfips'] == 0) & (population['Year'] <= 2018) & (population['Year'] > 2013)]
population = population.drop(
    columns=['IBRC_Geo_ID', 'Statefips', 'Countyfips', 'Population 0-4', 'Population 5-17', 'Population 18-24',
             'Population 25-44', 'Population 45-64', 'Population 65+'])
population = population.rename(columns={'Description': 'state', 'Year': 'year'})

In [9]:
population.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265 entries, 14 to 63880
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   state                265 non-null    object 
 1   year                 265 non-null    int64  
 2   Total Population     265 non-null    float64
 3   Population Under 18  265 non-null    float64
 4   Population 18-54     265 non-null    float64
 5   Population 55+       265 non-null    float64
 6   Male Population      265 non-null    float64
 7   Female Population    265 non-null    float64
dtypes: float64(6), int64(1), object(1)
memory usage: 18.6+ KB


In [10]:
# merge the population data with the incidents data

# Group incidents by year and state, calculate incident counts
incident_counts = incidents.groupby(['year', 'state']).size().reset_index(name='counts')

# Group population by year and state, summing the population
population_totals = population.groupby(['year', 'state'])['Total Population'].sum().reset_index()

# Merge incident counts and population totals on year and state
merged_data = pd.merge(incident_counts, population_totals, on=['year', 'state'], how='inner')

# Calculate Incident Density per 100k
merged_data['Incident Density per 100k'] = (merged_data['counts'] / merged_data['Total Population']) * 100000

# Append U.S. totals for each year
us_totals = merged_data.groupby('year').agg({'counts': 'sum', 'Total Population': 'sum'}).reset_index()
us_totals['state'] = 'U.S.'
us_totals['Incident Density per 100k'] = (us_totals['counts'] / us_totals['Total Population']) * 100000

# Append U.S. totals to the merged data
result = pd.concat([merged_data, us_totals]).reset_index(drop=True)[['year', 'state', 'Incident Density per 100k']]

incidents_density = pd.merge(incidents, result, on=['state', 'year'], how='inner')

In [11]:
incidents_density

,date,state,city_or_county,latitude,longitude,congressional_district,min_age_participants,avg_age_participants,max_age_participants,n_participants_child,...,n_killed,n_injured,n_arrested,n_unharmed,n_participants,year,month,day,category_index,Incident Density per 100k
0,2015-05-02,Indiana,Indianapolis,39.8322,-86.2492,7.0,19.0,19.0,19.0,0.0,...,0,1,0,0,1,2015,5,2,1,15.616436
1,2015-08-02,Indiana,Gary,41.5981,-87.3400,1.0,15.0,15.0,15.0,0.0,...,1,0,0,0,1,2015,8,2,1,15.616436
2,2015-04-15,Indiana,Indianapolis,39.7150,-86.2981,7.0,NaN,NaN,NaN,0.0,...,0,0,0,0,0,2015,4,15,0,15.616436
3,2015-12-18,Indiana,Evansville,37.9774,-87.5372,8.0,24.0,24.0,24.0,0.0,...,0,0,1,0,1,2015,12,18,0,15.616436
4,2015-05-14,Indiana,Gary,41.5982,-87.3488,1.0,32.0,46.0,59.0,0.0,...,1,0,0,1,2,2015,5,14,1,15.616436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216384,2018-03-24,Hawaii,Wailuku,NaN,NaN,NaN,26.0,26.0,26.0,0.0,...,0,0,1,0,1,2018,3,24,4,1.337470
216385,2018-01-07,Hawaii,Pahala,19.2000,-155.4800,2.0,33.0,33.0,33.0,0.0,...,0,0,1,0,1,2018,1,7,2,1.337470
216386,2018-03-05,Hawaii,Ocean View,19.1127,-155.7570,2.0,31.0,42.0,64.0,0.0,...,0,2,1,0,3,2018,3,5,1,1.337470
216387,2018-01-28,Hawaii,Waianae,21.4444,-158.1860,2.0,34.0,34.0,34.0,0.0,...,0,0,1,0,1,2018,1,28,2,1.337470


In [12]:
# save incidents_density to csv
incidents_density.to_csv("../data/extracted_features.csv", index=False)

In [13]:
gun ownership = pd.read_excel("../data/TL-354-State-Level Estimates of Household Firearm Ownership.xlsx", sheet_name=1)
csv = gun ownership.to_csv("../data/GunOwnership.csv", index = None, header=True)


In [13]:
# to integrate the data from the GunOwnership.csv file with the incidents_density dataframe

In [14]:
def preparing_df(df, df2, df3, df4, df5):
    df = df.drop(columns=['address', 'state_house_district', 'state_senate_district', 'participant_age1', 'participant_age_group1', 'participant_gender1'])
    df['date'] = pd.to_datetime(df['date'])
    df['year']= df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df = df.drop(columns=['date'])
    df = df[(df['year'] > 2013) & (df['year'] < 2019)]
    # controlli semantici
    cols = ['min_age_participants', 'max_age_participants', 'avg_age_participants', 'n_participants_child', 'n_participants_teen', 'n_participants_adult']
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    df['max_age_participants'] = np.where((df['max_age_participants'] < 0) | (df['max_age_participants'] > 115), np.nan, df['max_age_participants'])
    df['min_age_participants'] = np.where((df['min_age_participants'] < 0) | (df['min_age_participants'] > 115), np.nan,
                                          df['min_age_participants'])
    df['avg_age_participants'] = np.where((df['avg_age_participants'] < 0) | (df['avg_age_participants'] > 115), np.nan,
                                          df['avg_age_participants'])
    df['n_participants_child'] = np.where((df['n_participants_child']< 0) | (df['n_participants_child'] > df['n_participants']), np.nan,
                                          df['n_participants_child'])
    df['n_participants_teen'] = np.where((df['n_participants_teen'] < 0) | (df['n_participants_teen'] > df['n_participants']), np.nan,
                                          df['n_participants_teen'])
    df['n_participants_adult'] = np.where((df['n_participants_adult'] < 0) | (df['n_participants_adult'] > df['n_participants']), np.nan,
                                          df['n_participants_adult'])
    df['longitude'] = np.where(df['longitude'] > 0, df['longitude']*-1, df['longitude'])
    #21968 = San Bernardino shooting
    #88653 = Florida school shooting
    #116161 = Colorado, gangs
    #180038 = Boston, gangs
    #152656 = Texas church shooting
    #236261 = Orlando shooting
    df = df.drop([21968, 88653, 116161, 180038, 152656, 236261])
    df2 = df2[(df2['year'] > 2013) & (df2['year'] < 2019)]
    df3 = df3[(df3['year'] > 2013) & (df3['year'] < 2019)]
    df3['state'] = df3['state'].str.title()
    df4 = df4[(df4['Countyfips'] == 0) & (df4['Year'] > 2013) & (df4['Year'] < 2019)]
    df4 = df4.drop(
        columns=['IBRC_Geo_ID', 'Statefips', 'Countyfips', 'Population 0-4', 'Population 5-17', 'Population 18-24',
                 'Population 25-44', 'Population 45-64', 'Population 65+'])
    df4 = df4.rename(columns={'Year': 'year', 'Description': 'state'})
    df5 = df5[df5['Year'] > 2013]
    df5 = df5[['Year', 'STATE', 'HFR', 'universl', 'permit']]
    df5 = df5.rename(columns={'Year': 'year', 'STATE': 'state'})
    return df, df2, df3, df4, df5

In [14]:
df1 = incidents
df2 = poverty
df3 = districts
df4 = population
df5 = gun ownership

def incidents_to_win(df, df3):
    incidents_win = pd.DataFrame()
    years = [2014, 2015, 2016, 2017, 2018]
    for x in years:
        incidents = df[df['year'] == x]
        state_incidents = incidents['state'].value_counts().rename_axis('state').reset_index(name='counts')
        state_incidents['year'] = x
        year_results = df3
        if x % 2 == 0:
            year_results = year_results[year_results['year'] == x - 2]
            year_results['year'] = x
        else:
            year_results = year_results[year_results['year'] == x - 1]
            year_results['year'] = x
        year_results['win_percentage'] = (year_results['candidatevotes']/year_results['totalvotes'])*100
        year_results = year_results.drop(columns=['congressional_district', 'party', 'candidatevotes', 'totalvotes'])
        year_results = year_results.groupby('state', as_index=False).mean()
        temp = pd.merge(state_incidents, year_results)
        incidents_win = pd.concat([incidents_win, temp])
    incidents_win['incidents_to_win'] = incidents_win['counts']/incidents_win['win_percentage']
    incidents_win = incidents_win.drop(columns=['counts', 'win_percentage'])
    df = pd.merge(df, incidents_win, how='outer')
    return df

def involved_to_voters(df, df3):
    involved_voters = pd.DataFrame()
    years = [2014, 2015, 2016, 2017, 2018]
    for x in years:
        incidents = df[df['year'] == x]
        temp = incidents.groupby('state', as_index=False).sum()
        temp['n_involved'] = temp['n_killed'] + temp['n_injured']
        total_involved = temp[['state', 'n_involved']]
        total_involved['year'] = x
        year_results = df3
        if x % 2 == 0:
            year_results = year_results[year_results['year'] == x - 2]
            year_results['year'] = x
        else:
            year_results = year_results[year_results['year'] == x - 1]
            year_results['year'] = x
        year_results = year_results.groupby('state', as_index=False).sum()
        year_results = year_results.drop(columns=['congressional_district', 'party', 'year', 'totalvotes'])
        temp = pd.merge(total_involved, year_results)
        involved_voters = pd.concat([involved_voters, temp])
    involved_voters['involved_to_voters'] = (involved_voters['n_involved']/involved_voters['candidatevotes']) * 100000
    involved_voters = involved_voters.drop(columns=['n_involved', 'candidatevotes'])
    df = pd.merge(df, involved_voters, how='outer')
    return df

In [15]:
def new_features(df, df2, df3, df4, df5):
    df = crime_to_poverty(df, df2)
    df = average_age(df)
    df = average_participants(df)
    df = kill_to_gun(df, df5)
    df = poverty_to_gun(df, df2, df5)
    df = minors_percentage(df, df4)
    #df = incidents_to_win(df, df3)
    return df

def crime_to_poverty(df, df2):
    incidents_to_poverty = pd.DataFrame()
    months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    years = [2014, 2015, 2016, 2017, 2018]
    for x in years:
        incidents = df[df['year'] == x]
        for y in months:
            temp = incidents[incidents['month'] == y]
            state_incidents = temp['state'].value_counts().rename_axis('state').reset_index(name='counts')
            poverty_year = df2[df2['year'] == x]
            state_incidents['month'] = y
            temp = pd.merge(state_incidents, poverty_year, left_on='state', right_on='state', how='inner')
            incidents_to_poverty = pd.concat([incidents_to_poverty, temp])
    incidents_to_poverty['crime_ratio'] = incidents_to_poverty['counts']/incidents_to_poverty['povertyPercentage']
    incidents_to_poverty = incidents_to_poverty.drop(columns=['counts', 'povertyPercentage'])
    df = pd.merge(df, incidents_to_poverty, how='inner')
    return df

def average_age(df):
    avg_age = df.groupby('state', as_index=False)['avg_age_participants'].mean()
    avg_age = avg_age.rename(columns={'avg_age_participants': 'avg_age'})
    df = pd.merge(df, avg_age, how='inner')
    return df

def average_participants(df):
    avg_participants = df.groupby('state', as_index=False)['n_participants'].mean()
    avg_participants = avg_participants.rename(columns={'n_participants': 'avg_participants'})
    df = pd.merge(df, avg_participants, how='inner')
    return df

def kill_to_gun(df, df5):
    kills_to_gun = pd.DataFrame()
    months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    years = [2014, 2015, 2016]
    for x in years:
        incidents = df[df['year'] == x]
        for y in months:
            temp = incidents[incidents['month'] == y]
            temp = temp.groupby('state', as_index=False).sum()
            state_kills = temp[['state', 'n_killed']]
            state_kills['year'] = x
            state_kills['month'] = y
            gun_ownership = df5[df5['year']==x]
            temp = pd.merge(state_kills, gun_ownership, how='inner')
            kills_to_gun = pd.concat([kills_to_gun, temp])
    kills_to_gun['kill_to_gun'] = kills_to_gun['n_killed'] / (kills_to_gun['HFR']*100)
    kills_to_gun = kills_to_gun.drop(columns=['n_killed', 'HFR', 'universl', 'permit'])
    df = pd.merge(df, kills_to_gun, how='outer')
    return df

def minors_percentage(df, df4):
    minors = pd.DataFrame()
    years = [2014, 2015, 2016, 2017]
    for x in years:
        incidents = df[df['year'] == x]
        temp = incidents.groupby('state', as_index=False).sum()
        temp['n_participants_minors'] = temp['n_participants_child'] + temp['n_participants_teen']
        total_minors = temp[['state', 'n_participants_minors']]
        population_year = df4[df4['year']==x]
        temp = pd.merge(total_minors, population_year)
        minors = pd.concat([minors, temp])
    minors['minors_percentage'] = minors['n_participants_minors']/minors['Population Under 18']
    minors = minors.drop(columns=['n_participants_minors', 'Total Population', 'Population Under 18', 'Population 18-54',
                                  'Population 55+', 'Male Population', 'Female Population'])
    df = pd.merge(df, minors, how='outer')
    return df

def poverty_to_gun(df, df2, df5):
    temp = df5.rename(columns={'Year': 'year', 'STATE': 'state'})
    merge = pd.merge(df2, temp, how='inner')
    merge['poverty_to gun'] = merge['povertyPercentage']/(merge['HFR']*100)
    merge = merge.drop(columns=['povertyPercentage', 'HFR', 'universl', 'permit'])
    df = pd.merge(df, merge, how='outer')
    return df